# Prepare data for a analysis using CMS tools

- Baseando-se, entre outras coisas, no analysis grand challange: https://cms-opendata-workshop.github.io/workshop2024-lesson-remote-hackathon/06-agc.html


In [9]:
# First, install dependecies
!pip install -r ../requirements.txt


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [10]:
!pip install --upgrade pip

!pip install --upgrade awkward
!pip install --upgrade uproot
!pip install fsspec-xrootd
!pip install vector
!pip install --upgrade pandas
!pip install --upgrade coffea
!pip install --upgrade matplotlib

  Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.7/831.7 kB 6.9 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.1/633.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: awkward
    Found existing installation: awkward 1.10.3
    Uninstalling awkward-1.10.3:
      Successfully uninstalled awkward-1.10.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
coffea 0.7.21 requires awkward<2,>=1.10.3, but you have awkward 2.6.8 which is incompatible.
servicex 2.6.2 requires awkward<2,>=1.0.1, but you have awkward 2.6.8 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: uproot
    Found e

In [ ]:
## Besides the usual python packages, to do an modern analysis, we will need some additional tools.
## The most important one is the coffea package.

# - awkward
# - uproot
# - fsspec-xrootd
# - vector
# - coffea

In [9]:
import logging
import time

import awkward as ak
import cabinetry
import cloudpickle
import correctionlib
# from coffea import processor
# from coffea.nanoevents import NanoAODSchema
# from coffea.analysis_tools import PackedSelection
import copy
import hist
import matplotlib.pyplot as plt
import numpy as np
import pyhf
import requests
#import utils  # contains code for bookkeeping and cosmetics, as well as some boilerplate

logging.getLogger("cabinetry").setLevel(logging.INFO)

##
# The classics
import numpy as np
import matplotlib.pylab as plt
import matplotlib # To get the version

import pandas as pd

# The newcomers
import awkward as ak
import uproot

import vector
vector.register_awkward()

import requests
import os

import time

import json

## Choose a dataset through CMS Open Data Portal

- Access https://opendata.cern.ch/
- Choose a dataset
- Copy the link here (txt index)

In [17]:
## TXT index of the data set (root files)
file_name = "CMS_Run2016G_SinglePhoton_NANOAOD_UL2016_MiniAODv2_NanoAODv9-v2_2430000_file_index.txt"
data_set_link_txt = f'https://opendata.cern.ch/record/30531/files/{file_name}'


# Create a folder, if not exists
!mkdir data_txt

# Get data
!wget -iO $data_set_link_txt

# Move the file
!mv $file_name data_txt

mkdir: cannot create directory ‘data_txt’: File exists
--2024-09-18 18:13:58--  https://opendata.cern.ch/record/30531/files/CMS_Run2016G_SinglePhoton_NANOAOD_UL2016_MiniAODv2_NanoAODv9-v2_2430000_file_index.txt
Resolving opendata.cern.ch (opendata.cern.ch)... 137.138.6.31, 2001:1458:201:8b::100:1c8
Connecting to opendata.cern.ch (opendata.cern.ch)|137.138.6.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 918 [text/plain]
Saving to: ‘CMS_Run2016G_SinglePhoton_NANOAOD_UL2016_MiniAODv2_NanoAODv9-v2_2430000_file_index.txt’

CMS_Run2016G_Single 100%[===================>]     918  --.-KB/s    in 0s      

2024-09-18 18:13:59 (397 MB/s) - ‘CMS_Run2016G_SinglePhoton_NANOAOD_UL2016_MiniAODv2_NanoAODv9-v2_2430000_file_index.txt’ saved [918/918]

O: No such file or directory
No URLs found in O.
FINISHED --2024-09-18 18:13:59--
Total wall clock time: 1.1s
Downloaded: 1 files, 918 in 0s (397 MB/s)


In [18]:
# Read the file and split it into a list of root link strings

with open(f"data_txt/{file_name}", 'r', encoding='UTF-8') as file:
    root_files_list = file.read().splitlines()

root_files_list

['root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/28182352-6457-734D-995A-301A869F95B0.root',
 'root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/2C47055A-092A-3242-BEC4-6CA0B67557FA.root',
 'root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/37A146FF-19BC-4C4F-82D1-D0213E2BC798.root',
 'root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/6F892456-D392-4F43-8041-66B950DE0907.root',
 'root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/A962C9F2-A7C6-FB4B-9921-9E445A949F17.root',
 'root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/C50357CE-A9F6-EF4A-BFEB-2B6675B53C70.root']

In [19]:
## Run the loop for each file in the list

for filename in root_files_list:
    print(f"Opening...{filename}")



Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/28182352-6457-734D-995A-301A869F95B0.root
Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/2C47055A-092A-3242-BEC4-6CA0B67557FA.root
Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/37A146FF-19BC-4C4F-82D1-D0213E2BC798.root
Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/6F892456-D392-4F43-8041-66B950DE0907.root
Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/A962C9F2-A7C6-FB4B-9921-9E445A949F17.root
Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/C50357CE-A9F6-EF4A-BFEB-2B6675B53C70.root


In [25]:
import subprocess

file_name = "Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt"
subprocess.run(["wget", f"https://opendata.cern.ch/record/14220/files/{file_name}"])

--2024-09-18 18:42:04--  https://opendata.cern.ch/record/14220/files/Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt
Resolving opendata.cern.ch (opendata.cern.ch)... 137.138.6.31, 2001:1458:201:8b::100:1c8
Connecting to opendata.cern.ch (opendata.cern.ch)|137.138.6.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11686 (11K) [text/plain]
Saving to: ‘Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt’

     0K .......... .                                          100% 19.4M=0.001s

2024-09-18 18:42:05 (19.4 MB/s) - ‘Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt’ saved [11686/11686]



CompletedProcess(args=['wget', 'https://opendata.cern.ch/record/14220/files/Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt'], returncode=0)

In [40]:

#########################################################################################
def build_lumi_mask(lumifile, tree, verbose=False):
    # lumifile should be the name/path of the file

    '''All CMS data is studied by data quality monitoring groups for various subdetectors to determine whether it is suitable for physics analysis. Data can be accepted or rejected in units of “luminosity sections”. 
    These are periods of time covering 2**18  LHC revolutions, or about 23 seconds. 
    The list of validated runs and luminosity sections is stored in a json file that can be downloaded from the Open Data Portal.
    First, obtain the file with the list of validated runs and luminosity sections for 2016 data:
    this file can be download from: wget https://opendata.cern.ch/record/14220/files/Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt'''
    import subprocess
    lumifile = "Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt"
    subprocess.run(["wget", f"https://opendata.cern.ch/record/14220/files/{file_name}"])

    good_luminosity_sections = ak.from_json(open(lumifile, 'rb'))

    # Pull out the good runs as integers
    good_runs = np.array(good_luminosity_sections.fields).astype(int)
    #good_runs

    # Get the good blocks as an awkward array
    # First loop over to get them as a list
    all_good_blocks = []
    for field in good_luminosity_sections.fields:
        all_good_blocks.append(good_luminosity_sections[field])

    # Turn the list into an awkward array
    all_good_blocks = ak.Array(all_good_blocks)
    all_good_blocks[11]

    # Assume that tree is a NanoAOD Events tree
    nevents = tree.num_entries
    if verbose:
        print(f"nevents: {nevents}")
        print()
        print("All good runs")
        print(good_runs)
        print()
        print("All good blocks")
        print(all_good_blocks)
        print()

    # Get the runs and luminosity blocks from the tree
    run = tree['run'].array()
    lumiBlock = tree['luminosityBlock'].array()

    if verbose:
        print("Runs from the tree")
        print(run)
        print()
        print("Luminosity blocks from the tree")
        print(lumiBlock)
        print()

    # ChatGPT helped me with this part!
    # Find index of values in arr2 if those values appear in arr1

    def find_indices(arr1, arr2):
        index_map = {value: index for index, value in enumerate(arr1)}
        return [index_map.get(value, -1) for value in arr2]

    # Get the indices that say where the good runs are in the lumi file
    # for the runs that appear in the tree
    good_runs_indices = find_indices(good_runs, run)

    # For each event, calculate the difference between the luminosity block for that event
    # and the good luminosity blocks for that run for that event
    diff = lumiBlock - all_good_blocks[good_runs_indices]

    if verbose:
        print("difference between event lumi blocks and the good lumi blocks")
        print(diff)
        print()

    # If the lumi block appears between any of those good block numbers, 
    # then one difference will be positive and the other will be negative
    # 
    # If it it outside of the range, both differences will be positive or 
    # both negative.
    #
    # The product will be negagive if the lumi block is in the range
    # and positive if it is not in the range
    prod_diff = ak.prod(diff, axis=2)

    if verbose:
        print("product of the differences")
        print(prod_diff)
        print()

    mask = ak.any(prod_diff<=0, axis=1)

    return mask


## Process the files

list_of_event_data = ['FatJet_msoftdrop', 'FatJet_particleNet_TvsQCD', 'FatJet_tau2', 'FatJet_tau3', 'FatJet_pt', 'FatJet_eta', 'FatJet_phi', 'FatJet_mass', # FatJet
    'Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_mass', 'Muon_miniIsoId', 'Muon_tightId', # Muons
    'Jet_btagDeepB', 'Jet_jetId', 'Jet_pt', 'Jet_eta', 'Jet_phi', 'Jet_mass', # Jets
    'PuppiMET_pt', 'PuppiMET_phi'] # MET]


def select_events(filename: str, dataset='default', IS_DATA=False, list_of_event_data=[]):

    '''
    filename: the name of an input NanoAOD ROOT file. URL or location from the single .root file, to be accessed through uproot.open() method.
    dataset: string to be used to organize output files
    IS_DATA: a flag that should be set to `True` if the input datafile is *collision* data.


    This function saves 
    * Mass of the $t\overline{t}$ system
    * $p_T$ of the muon
    * $\eta$ of the muon
    * `pileup`
    * `weight`
    * `nevents`
    * `N_gen`
    * `gw_pos`
    * `gw_neg`

    This function will process a single root file into a .csv format with some key data extracted.
    It can be modified to extract different information from the root file as well.'''
    
    print(f"Opening...{filename}")
    
    try:
        f = uproot.open(filename)
    except:
        print(f"Could not open {filename}")
        return None
        
    events = f['Events']

    nevents = events.num_entries

    print(f"{nevents = }")

    # Get selected events as an array

    selected_events = {}
    for event_data in list_of_event_data:
        selected_events[event_data] = events[event_data].array()
                          

    # # MET ---------------------------------------------------------
    # met_pt = events['PuppiMET_pt'].array()
    selected_events['PuppiMET_eta']  = 0*events['PuppiMET_pt'].array()  # Fix this to be 0
    # met_phi = events['PuppiMET_phi'].array() 

    selected_events['ht_lep'] = selected_events['Muon_pt'] + selected_events['PuppiMET_pt']

    return selected_events


def apply_cuts(events, selected_events, cuts = True, IS_DATA=False):
    #####################################################################################
    # Cuts
    #####################################################################################
    if cuts:
        print("Applying cuts")
        # Particle-specific cuts --------------------------------------
        tau32 = selected_events['FatJet_tau3']/selected_events['FatJet_tau2']

        #cut_fatjet = (tau32>0.67) & (fatjet_eta>-2.4) & (fatjet_eta<2.4) & (fatjet_mSD>105) & (fatjet_mSD<220)
        cut_fatjet = (selected_events['FatJet_pt'] > 500) & (selected_events['FatJet_particleNet_TvsQCD'] > 0.5)

        cut_muon = (selected_events['Muon_pt'] > 55) & (selected_events['Muon_eta']>-2.4) & (selected_events['Muon_eta']<2.4) & \
                (selected_events['Muon_tightId'] == True) & (selected_events['Muon_miniIsoId']>1) & (selected_events['ht_lep']>150)

        cut_jet = (selected_events['Jet_btagDeepB'] > 0.5) & (selected_events['Jet_jetId']>=4)



        # Event cuts -------------------------------------------------
        cut_met = (selected_events['PuppiMET_pt'] > 50)

        cut_nmuons = ak.num(cut_muon[cut_muon]) == 1
        cut_njets = ak.num(cut_jet[cut_jet]) == 1


        cut_trigger = (events['HLT_TkMu50'].array())

        cut_ntop = ak.num(cut_fatjet[cut_fatjet]) == 1

        cut_full_event = None
        if IS_DATA:    
            mask_lumi = build_lumi_mask('Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt', events)#, verbose=True)
            cut_full_event = cut_trigger & cut_nmuons & cut_met & cut_ntop & mask_lumi
        else:
            cut_full_event = cut_trigger & cut_nmuons & cut_met & cut_ntop
        
        # Apply the cuts and calculate the di-top mass
        fatjets = ak.zip(
            {"pt": selected_events['FatJet_pt'][cut_full_event][cut_fatjet[cut_full_event]], 
            "eta": selected_events['FatJet_eta'][cut_full_event][cut_fatjet[cut_full_event]], 
            "phi": selected_events['FatJet_phi'][cut_full_event][cut_fatjet[cut_full_event]], 
            "mass": selected_events['FatJet_mass'][cut_full_event][cut_fatjet[cut_full_event]]},
            with_name="Momentum4D",
        )

        muons = ak.zip(
            {"pt": selected_events['Muon_pt'][cut_full_event][cut_muon[cut_full_event]], 
            "eta": selected_events['Muon_eta'][cut_full_event][cut_muon[cut_full_event]], 
            "phi": selected_events['Muon_phi'][cut_full_event][cut_muon[cut_full_event]], 
            "mass": selected_events['Muon_mass'][cut_full_event][cut_muon[cut_full_event]]},
            with_name="Momentum4D",
        )

        jets = ak.zip(
            {"pt": selected_events['Jet_pt'][cut_full_event][cut_jet[cut_full_event]], 
            "eta": selected_events['Jet_eta'][cut_full_event][cut_jet[cut_full_event]], 
            "phi": selected_events['Jet_phi'][cut_full_event][cut_jet[cut_full_event]], 
            "mass": selected_events['Jet_mass'][cut_full_event][cut_jet[cut_full_event]]},
            with_name="Momentum4D",
        )

        met = ak.zip(
            {"pt": selected_events['PuppiMET_pt'][cut_full_event], 
            "eta": selected_events['PuppiMET_eta'][cut_full_event], 
            "phi": selected_events['PuppiMET_phi'][cut_full_event], 
            "mass": 0}, # We assume this is a neutrino with 0 mass
            with_name="Momentum4D",
        )

        p4mu, p4fj, p4j, p4met = ak.unzip(ak.cartesian([muons, fatjets, jets, met]))
        
        p4tot = p4mu + p4fj + p4j + p4met
        
        # Shape the weights and pileup
        N_gen = -999
        pileup = -999
        gw_pos = -999
        gw_neg = -999

        pileup_per_candidate = None
        
        tmpval_events = np.ones(len(ak.flatten(p4tot.mass)))
        tmpval = ak.ones_like(p4tot.mass)


        # Put in the MC weights
        if not IS_DATA:
            gen_weights = events['genWeight'].array()[cut_full_event]
            pileup = events['Pileup_nTrueInt'].array()[cut_full_event]

            gen_weights_per_candidate = tmpval * gen_weights
            #print(gen_weights_per_candidate)

            pileup_per_candidate = tmpval * pileup
            #print(pileup_per_candidate)

            # Get values associated with the total number of events. 
            # It's going to duplicate the number of entries, but we'll save the same value to 
            # each event
            gen_weights_org = events['genWeight'].array()

            gw_pos = ak.count(gen_weights_org[gen_weights_org > 0])
            gw_neg = ak.count(gen_weights_org[gen_weights_org < 0])
            N_gen = gw_pos - gw_neg
        else:
            pileup_per_candidate = -999*tmpval
            gen_weights_per_candidate = -999*tmpval
        

        # Build a dictionary and dataframe to write out the subset of data
        # we are interested in
        mydict = {}
        mydict['mtt'] = ak.flatten(p4tot.mass) 
        mydict['mu_pt'] = ak.flatten(p4mu.pt) 
        mydict['mu_abseta'] = np.abs(ak.flatten(p4mu.eta))
        mydict['pileup'] = ak.flatten(pileup_per_candidate)
        mydict['weight'] = ak.flatten(gen_weights_per_candidate)
        mydict['nevents'] = nevents*tmpval_events
        mydict['N_gen'] = N_gen*tmpval_events
        mydict['gw_pos'] = gw_pos*tmpval_events
        mydict['gw_neg'] = gw_neg*tmpval_events

        df = pd.DataFrame.from_dict(mydict)

        outfilename = f"OUTPUT_{dataset}_{filename.split('/')[-1].split('.')[0]}.csv"
        print(f'Saving output to {outfilename}')

        df.to_csv(outfilename, index=False)

        return df

In [37]:
test_file = 'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/120000/0BD60695-8388-5141-B157-32AE1A3B4885.root'



In [41]:
select_events = select_events(test_file, IS_DATA=False, list_of_event_data=list_of_event_data)

Opening...root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/120000/0BD60695-8388-5141-B157-32AE1A3B4885.root
nevents = 1365000


In [43]:
select_events['Muon_mass']

<Array [[], [0.106], [0.106], ..., [], [], []] type='1365000 * var * float32'>

In [7]:
# Download these files

for i, datasetname in enumerate(root_files_list):
    
    print(datasetname)

    outfilename = f'FILE_LIST_{datasetname}.txt'

    # Remove the file if it exists
    # try:
    #     os.remove(outfilename)
    # except OSError:
    #     pass
    url = datasetname
    #for url in root_files_list[i]:
    #print(url)

    r = requests.get(url, allow_redirects=True)

    open(outfilename, 'a').write(r.text)

root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16MiniAODv2/ZPrime2DarkPhoton_HMass-1000_DPMass-0p3_TuneCP5_13TeV-pythia8/MINIAODSIM/106X_mcRun2_asymptotic_v17-v2/30000/1EACD6D9-1EEE-B548-B47F-36E9641118C4.root


InvalidSchema: No connection adapters were found for 'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16MiniAODv2/ZPrime2DarkPhoton_HMass-1000_DPMass-0p3_TuneCP5_13TeV-pythia8/MINIAODSIM/106X_mcRun2_asymptotic_v17-v2/30000/1EACD6D9-1EEE-B548-B47F-36E9641118C4.root'

In [ ]:
##

In [ ]:
#